In [45]:
import mne
import numpy as np

import os

from glob import glob
from tqdm import tqdm
from utils import read_TUH_edf
import pickle
import torch

In [19]:
dir_path = "/scratch/agjma/tuh_selected/"

edf_files = glob(os.path.join(dir_path, "*.edf"))

sample_file = edf_files[1]

In [20]:
## process file
channel_order = [
                'Fp1', 'Fp2',
        'F7', 'F3', 'Fz', 'F4', 'F8',
        'T3', 'C3', 'Cz', 'C4', 'T4',
        'T5', 'P3', 'Pz', 'P4', 'T6',
                 'O1', 'O2'
    ]


raw = read_TUH_edf(sample_file)

try:
    raw.pick(channel_order)
    raw.reorder_channels(channel_order)
except:
    pass

In [21]:
raw

<RawEDF | aaaaabot_s002_t002.edf, 19 x 88064 (344.0 s), ~12.8 MB, data loaded>

In [48]:
# we now want to take random 60 sec samples from the data
n_samples = 2
sample_length = 60 # seconds
sample_freq = raw.info['sfreq']

samples = []

for i in range(n_samples):
    # because the sample is 60 seconds long, we need to take a random start time that is at least 60 seconds from the end
    start_time = np.random.randint(0, raw.times[-1] - sample_length)
    sample = raw.copy().crop(tmin=start_time, tmax=start_time + sample_length)

    x = torch.from_numpy(sample.get_data())

    # save the sample as pickle file
    


torch.Size([19, 15361])
torch.Size([19, 15361])


In [28]:
256*300

76800

In [40]:
raw.times[-1]

343.99609375

In [30]:
raw.n_times

88064

In [52]:
raw.filenames[0]

'/scratch/agjma/tuh_selected/aaaaabot_s002_t002.edf'

In [56]:
os.path.basename(sample_file)
save_dir = "/scratch/s194101/random_tuh_selected_samples_60/"
save_file = os.path.join(save_dir, os.path.basename(sample_file).replace(".edf", f"_sample_{i}.pkl"))
save_file

'/scratch/s194101/random_tuh_selected_samples_60/aaaaabot_s002_t002_sample_1.pkl'

In [ ]:
def process_file(file_path, n_samples, sample_length, save_dir):
    channel_order = [
                'Fp1', 'Fp2',
        'F7', 'F3', 'Fz', 'F4', 'F8',
        'T3', 'C3', 'Cz', 'C4', 'T4',
        'T5', 'P3', 'Pz', 'P4', 'T6',
                 'O1', 'O2'
    ]
    
    raw = read_TUH_edf(file_path)
    try:
        raw.pick(channel_order)
        raw.reorder_channels(channel_order)
    except:
        pass

    sample_freq = raw.info['sfreq']

    for i in range(n_samples):
        # because the sample is 60 seconds long, we need to take a random start time that is at least 60 seconds from the end
        start_time = np.random.randint(0, raw.times[-1] - sample_length)
        sample = raw.copy().crop(tmin=start_time, tmax=start_time + sample_length)

        x = torch.from_numpy(sample.get_data())

        # save the sample as pickle file
        save_file = os.path.join(save_dir, os.path.basename(file_path).replace(".edf", f"_sample_{i}.pkl"))
        with open(save_file, 'wb') as f:
            pickle.dump(x, f, protocol=pickle.HIGHEST_PROTOCOL)

        
    

In [ ]:
for file_path in tqdm(edf_files):
    save_dir = "/scratch/s194101/random_tuh_selected_samples_60/"
    process_file(file_path, n_samples=2, sample_length=60, save_dir=save_dir)